<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1DvSkJZaHteEmpSt21GCElS0fVxhXNGYR?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions

Learn by building. Get expert mentorship and work on real AI projects.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




### **AgentScope: Agent-Oriented Programming for Building LLM Applications**

### **Installation**

In [13]:
!pip install agentscope pydantic

###**Imports**

In [21]:
from agentscope.agent import ReActAgent, AgentBase
from agentscope.formatter import OpenAIChatFormatter
from agentscope.memory import InMemoryMemory
from agentscope.message import Msg
from agentscope.model import OpenAIChatModel
import asyncio
import os
from google.colab import userdata
import os
from agentscope.tool import Toolkit, execute_python_code
from pydantic import BaseModel, Field
import json
from agentscope.message import Msg
from agentscope.pipeline import MsgHub

### **Configure API Keys**

In [16]:
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

### **Tool Usage and Creating ReAct Agent**

In [11]:
async def creating_react_agent() -> None:
    """Create a ReAct agent and run a simple task."""
    # Prepare tools
    toolkit = Toolkit()
    toolkit.register_tool_function(execute_python_code)

    jojo= ReActAgent(
        name="Jojo",
        sys_prompt="You're a helpful assistant named Jojo",
        model=OpenAIChatModel(
            model_name="gpt-4o-mini",
            api_key=api_key,
            stream=True,

        ),
        formatter= OpenAIChatFormatter(),
        toolkit=toolkit,
        memory=InMemoryMemory(),
    )

    msg = Msg(
        name="user",
        content="Hi! Jojo, run bubble sorting program in Python.",
        role="user",
    )
    response = await jojo(msg)   # get response
    print("🤖 Jojo:", response.content)  # show reply
await creating_react_agent()


Jojo: {
    "type": "tool_use",
    "id": "call_6ypG7QNr6eR3F4UGXktHF8V9",
    "name": "execute_python_code",
    "input": {
        "code": "def bubble_sort(arr):\n    n = len(arr)\n    # Traverse through all array elements\n    for i in range(n):\n        # Last i elements are already in place\n        for j in range(0, n-i-1):\n            # Traverse the array from 0 to n-i-1\n            # Swap if the element found is greater\n            # than the next element\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1], arr[j]\n    return arr\n\n# Example usage:\narr = [64, 34, 25, 12, 22, 11, 90]\nsorted_arr = bubble_sort(arr)\nprint(sorted_arr)"
    }
}
system: {
    "type": "tool_result",
    "id": "call_6ypG7QNr6eR3F4UGXktHF8V9",
    "name": "execute_python_code",
    "output": [
        {
            "type": "text",
            "text": "<returncode>0</returncode><stdout>[11, 12, 22, 25, 34, 64, 90]\n</stdout><stderr></stderr>"
        }
    ]
}
Jojo: The 

 ### **Structured Output Using Pydantic**

In [20]:
# Create an ReAct agent
agent = ReActAgent(
    name="Jarvis",
    sys_prompt="You're a helpful assistant named Jarvis.",
    model=OpenAIChatModel(
            model_name="gpt-4o-mini",
            api_key=api_key,
            stream=True,

        ),
        formatter= OpenAIChatFormatter(),
    toolkit=Toolkit(),
    parallel_tool_calls=True,
)


# The structured model
class Model(BaseModel):
    name: str = Field(description="The name of the person")
    description: str = Field(
        description="A one-sentence description of the person",
    )
    age: int = Field(description="The age")
    honor: list[str] = Field(description="A list of honors of the person")


async def example_structured_output() -> None:
    """The example structured output"""
    res = await agent(
        Msg(
            "user",
            "Introduce Einstein",
            "user",
        ),
        structured_model=Model,
    )

    print("\nThe structured output:")
    print(json.dumps(res.metadata, indent=4))


await example_structured_output()

Jarvis: Albert Einstein was a theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (the other being quantum mechanics). His work revolutionized our understanding of space, time, and energy, leading to groundbreaking advancements in various scientific fields. Einstein is best known for his famous equation E=mc², which expresses the equivalence of mass and energy. He was awarded the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect, providing significant evidence for the particle theory of light.

The structured output:
{
    "name": "Albert Einstein",
    "description": "One of the most influential physicists of the 20th century, known for developing the theory of relativity.",
    "age": 76,
    "honor": [
        "Nobel Prize in Physics (1921)",
        "Copley Medal (1925)",
        "Max Planck Medal (1929)"
    ]
}


### **Message Sharing**

In multi-agent conversation, exchanging messages explicitly may not be efficient and convenient, especially when broadcasting messages among multiple agents

In [23]:
model=OpenAIChatModel(model_name="gpt-4o-mini", api_key=api_key)
formatter= OpenAIChatFormatter()

alice = ReActAgent(
    name="Alice",
    sys_prompt="You're a student named Alice.",
    model=model,
    formatter=formatter,
    toolkit=Toolkit(),
    memory=InMemoryMemory(),
)

bob = ReActAgent(
    name="Bob",
    sys_prompt="You're a student named Bob.",
    model=model,
    formatter=formatter,
    toolkit=Toolkit(),
    memory=InMemoryMemory(),
)

charlie = ReActAgent(
    name="Charlie",
    sys_prompt="You're a student named Charlie.",
    model=model,
    formatter=formatter,
    toolkit=Toolkit(),
    memory=InMemoryMemory(),
)


async def example_msghub() -> None:
    """Example of using MsgHub for multi-agent conversation."""
    async with MsgHub(
        [alice, bob, charlie],
        announcement=Msg(
            "system",
            "Now you meet each other with a brief self-introduction.",
            "system",
        ),
    ) as hub:
        await alice()
        await bob()
        await charlie()




await(example_msghub())

Alice: Hello! I'm Alice, a student currently pursuing my studies. I have a keen interest in learning new things and engaging in discussions on a variety of topics. I enjoy reading, exploring new ideas, and sharing knowledge with others. Nice to meet you!
Bob: Hi, I'm Bob! I'm a student with a passion for learning and exploring new subjects. I'm particularly interested in science and technology, but I enjoy delving into literature and arts as well. I love collaborating with others and sharing ideas. Looking forward to our interactions!
Charlie: Hi everyone! I'm Charlie, a fellow student, and I'm excited to connect with you all. I have a diverse range of interests, including technology, literature, and sports. I love diving deep into subjects that pique my curiosity and sharing insights with friends. Can't wait to learn and grow together!

=== Conversation Log ===
